# Google Drive Migration Helper

이 노트북을 사용하여 개인 Google 계정에서 회사 Google 계정으로 파일을 마이그레이션할 수 있습니다.

## 사용 전 준비사항
1. Google Cloud Console에서 OAuth 2.0 클라이언트 ID 생성
2. `credentials.json` 파일을 이 노트북과 같은 폴더에 업로드
3. 마이그레이션할 파일들이 개인 계정에 있는지 확인

## 주의사항
- 마이그레이션은 되돌릴 수 없으므로 신중하게 진행하세요
- 대용량 파일의 경우 시간이 오래 걸릴 수 있습니다
- 네트워크 연결이 안정적인 환경에서 실행하세요


In [ ]:
# 필요한 라이브러리 설치 및 임포트
%pip install google-api-python-client google-auth-oauthlib google-auth-httplib2
%pip install pandas tqdm python-dotenv

import os
import json
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from datetime import datetime

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

print("✅ 라이브러리 로드 완료")


In [ ]:
# Google Drive API 설정
SCOPES = [
    'https://www.googleapis.com/auth/drive',
    'https://www.googleapis.com/auth/drive.file',
    'https://www.googleapis.com/auth/drive.metadata',
    'https://www.googleapis.com/auth/drive.readonly'
]

# 파일 설정
CREDENTIALS_FILE = 'credentials.json'
PERSONAL_TOKEN_FILE = 'personal_token.json'
WORK_TOKEN_FILE = 'work_token.json'

# 마이그레이션 설정
MAX_FILE_SIZE = 5 * 1024 * 1024 * 1024  # 5GB
BATCH_SIZE = 100
MAX_RETRIES = 3

print("✅ 설정 완료")


In [ ]:
def authenticate_google_account(credentials_file, token_file, account_name):
    """
    Google 계정 인증
    
    Args:
        credentials_file: 인증 파일 경로
        token_file: 토큰 저장 파일 경로
        account_name: 계정 이름 (표시용)
    
    Returns:
        googleapiclient.discovery.Resource: Google Drive API 서비스
    """
    creds = None
    
    # 기존 토큰 로드
    if os.path.exists(token_file):
        creds = Credentials.from_authorized_user_file(token_file, SCOPES)
    
    # 토큰이 없거나 유효하지 않은 경우
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            # 토큰 갱신
            creds.refresh(Request())
        else:
            # 새 인증 플로우
            flow = InstalledAppFlow.from_client_secrets_file(credentials_file, SCOPES)
            creds = flow.run_local_server(port=0)
        
        # 토큰 저장
        with open(token_file, 'w') as token:
            token.write(creds.to_json())
    
    # Google Drive API 서비스 생성
    service = build('drive', 'v3', credentials=creds)
    
    # 사용자 정보 확인
    try:
        about = service.about().get(fields='user').execute()
        user_info = about.get('user', {})
        print(f"✅ {account_name} 계정 인증 완료: {user_info.get('displayName', 'Unknown')} ({user_info.get('emailAddress', 'Unknown')})")
    except Exception as e:
        print(f"❌ {account_name} 계정 사용자 정보 조회 실패: {e}")
    
    return service

print("✅ 인증 함수 정의 완료")


In [ ]:
# 계정 인증 실행
print("🔐 Google 계정 인증을 시작합니다...")
print("=" * 50)

# 개인 계정 인증
print("1️⃣ 개인 Google 계정 인증")
personal_service = authenticate_google_account(CREDENTIALS_FILE, PERSONAL_TOKEN_FILE, "개인")

print("\n" + "=" * 50)

# 회사 계정 인증  
print("2️⃣ 회사 Google 계정 인증")
work_service = authenticate_google_account(CREDENTIALS_FILE, WORK_TOKEN_FILE, "회사")

print("\n✅ 모든 계정 인증이 완료되었습니다!")


In [ ]:
def scan_drive_files(service, folder_id='root', folder_name='Root'):
    """
    Google Drive 파일 스캔
    
    Args:
        service: Google Drive API 서비스
        folder_id: 스캔할 폴더 ID (기본값: 'root')
        folder_name: 폴더 이름 (표시용)
    
    Returns:
        list: 파일 정보 리스트
    """
    files = []
    
    try:
        # 폴더 내 파일 목록 조회
        query = f"'{folder_id}' in parents and trashed=false"
        
        results = service.files().list(
            q=query,
            pageSize=1000,
            fields="nextPageToken, files(id, name, mimeType, size, createdTime, modifiedTime, parents, shared)"
        ).execute()
        
        items = results.get('files', [])
        
        for item in items:
            file_info = {
                'id': item['id'],
                'name': item['name'],
                'mimeType': item.get('mimeType', ''),
                'size': int(item.get('size', 0)) if item.get('size') else 0,
                'createdTime': item.get('createdTime', ''),
                'modifiedTime': item.get('modifiedTime', ''),
                'parents': item.get('parents', []),
                'shared': item.get('shared', False),
                'isFolder': item.get('mimeType') == 'application/vnd.google-apps.folder'
            }
            files.append(file_info)
        
        print(f"📁 {folder_name} 폴더에서 {len(files)}개 항목을 찾았습니다.")
        
    except HttpError as error:
        print(f"❌ 파일 스캔 중 오류 발생: {error}")
    
    return files

def format_file_size(size_bytes):
    """파일 크기 포맷팅"""
    if size_bytes == 0:
        return "0B"
    
    size_names = ["B", "KB", "MB", "GB", "TB"]
    i = 0
    while size_bytes >= 1024 and i < len(size_names) - 1:
        size_bytes /= 1024.0
        i += 1
    
    return f"{size_bytes:.1f}{size_names[i]}"

print("✅ 파일 스캔 함수 정의 완료")


In [ ]:
# 개인 계정 파일 스캔
print("🔍 개인 계정 파일을 스캔합니다...")
personal_files = scan_drive_files(personal_service, 'root', '개인 계정')

# 파일 정보 요약
if personal_files:
    df_personal = pd.DataFrame(personal_files)
    
    print(f"\n📊 개인 계정 파일 요약:")
    print(f"  - 총 파일 수: {len(df_personal)}")
    print(f"  - 폴더 수: {df_personal['isFolder'].sum()}")
    print(f"  - 파일 수: {(~df_personal['isFolder']).sum()}")
    
    if not df_personal['isFolder'].all():
        total_size = df_personal[~df_personal['isFolder']]['size'].sum()
        print(f"  - 총 크기: {format_file_size(total_size)}")
    
    # 파일 타입별 분포
    print(f"\n📋 파일 타입별 분포:")
    mime_counts = df_personal['mimeType'].value_counts().head(10)
    for mime_type, count in mime_counts.items():
        print(f"  {mime_type}: {count}개")
    
    # 공유 파일 수
    shared_count = df_personal['shared'].sum()
    print(f"\n🔗 공유 중인 파일: {shared_count}개")
    
    # 대용량 파일 확인
    large_files = df_personal[df_personal['size'] > 100 * 1024 * 1024]  # 100MB 이상
    if not large_files.empty:
        print(f"\n📦 대용량 파일 (100MB 이상): {len(large_files)}개")
        for _, file in large_files.iterrows():
            print(f"  - {file['name']}: {format_file_size(file['size'])}")
else:
    print("❌ 개인 계정에서 파일을 찾을 수 없습니다.")


In [ ]:
def migrate_file(source_service, target_service, file_info, target_folder_id='root'):
    """
    단일 파일 마이그레이션
    
    Args:
        source_service: 소스 Google Drive API 서비스
        target_service: 타겟 Google Drive API 서비스
        file_info: 파일 정보 딕셔너리
        target_folder_id: 타겟 폴더 ID
    
    Returns:
        dict: 마이그레이션 결과
    """
    result = {
        'file_id': file_info['id'],
        'file_name': file_info['name'],
        'success': False,
        'error': None,
        'new_file_id': None
    }
    
    try:
        if file_info['isFolder']:
            # 폴더 생성
            folder_metadata = {
                'name': file_info['name'],
                'mimeType': 'application/vnd.google-apps.folder',
                'parents': [target_folder_id]
            }
            
            new_folder = target_service.files().create(
                body=folder_metadata,
                fields='id'
            ).execute()
            
            result['success'] = True
            result['new_file_id'] = new_folder.get('id')
            
        else:
            # 파일 복사 (Google Drive API v3의 올바른 방법)
            file_metadata = {
                'name': file_info['name'],
                'parents': [target_folder_id]
            }
            
            # files().copy() 메서드 사용
            new_file = target_service.files().copy(
                fileId=file_info['id'],
                body=file_metadata,
                fields='id'
            ).execute()
            
            result['success'] = True
            result['new_file_id'] = new_file.get('id')
    
    except HttpError as error:
        result['error'] = str(error)
    except Exception as error:
        result['error'] = str(error)
    
    return result

def migrate_files_batch(source_service, target_service, files_df, target_folder_id='root'):
    """
    파일 배치 마이그레이션
    
    Args:
        source_service: 소스 Google Drive API 서비스
        target_service: 타겟 Google Drive API 서비스
        files_df: 파일 정보 DataFrame
        target_folder_id: 타겟 폴더 ID
    
    Returns:
        list: 마이그레이션 결과 리스트
    """
    results = []
    
    print(f"🚀 마이그레이션을 시작합니다. 총 {len(files_df)}개 파일을 처리합니다.")
    
    # 진행률 표시를 위한 tqdm 설정
    for idx, (_, file_info) in enumerate(tqdm(files_df.iterrows(), total=len(files_df), desc="마이그레이션 진행")):
        result = migrate_file(source_service, target_service, file_info, target_folder_id)
        results.append(result)
        
        # 진행 상황 출력
        if result['success']:
            print(f"✅ {file_info['name']} - 성공")
        else:
            print(f"❌ {file_info['name']} - 실패: {result['error']}")
        
        # API 할당량 관리를 위한 잠시 대기
        if (idx + 1) % 10 == 0:
            import time
            time.sleep(1)
    
    return results

print("✅ 마이그레이션 함수 정의 완료")


In [ ]:
# 마이그레이션 설정 및 실행
print("⚙️ 마이그레이션 설정을 확인합니다.")
print(f"  - 최대 파일 크기: {format_file_size(MAX_FILE_SIZE)}")
print(f"  - 배치 크기: {BATCH_SIZE}")
print(f"  - 최대 재시도 횟수: {MAX_RETRIES}")

# 마이그레이션할 파일 필터링
if personal_files:
    df_personal = pd.DataFrame(personal_files)
    
    # 마이그레이션 가능한 파일 필터링
    migratable_files = df_personal[
        (df_personal['size'] <= MAX_FILE_SIZE) | 
        (df_personal['isFolder'] == True)
    ]
    
    print(f"\n✅ 마이그레이션 가능한 파일: {len(migratable_files)}개")
    
    # 마이그레이션 불가능한 파일 확인
    too_large_files = df_personal[
        (df_personal['size'] > MAX_FILE_SIZE) & 
        (df_personal['isFolder'] == False)
    ]
    
    if not too_large_files.empty:
        print(f"\n⚠️ 마이그레이션 불가능한 파일 (크기 초과): {len(too_large_files)}개")
        for _, file in too_large_files.iterrows():
            print(f"  - {file['name']}: {format_file_size(file['size'])}")
else:
    print("❌ 마이그레이션할 파일이 없습니다.")
    migratable_files = pd.DataFrame()


In [ ]:
# 마이그레이션 실행 (주의: 실제 실행하려면 아래 주석 해제)
print("⚠️ 마이그레이션을 실행하시겠습니까?")
print("주의: 이 작업은 되돌릴 수 없습니다!")
print("\n실행하려면 아래 셀의 주석을 해제하고 실행하세요.")

# 실행하려면 아래 주석을 해제하세요
# if not migratable_files.empty:
#     print("🚀 마이그레이션을 시작합니다...")
#     results = migrate_files_batch(personal_service, work_service, migratable_files)
#     
#     # 결과 요약
#     success_count = sum(1 for r in results if r['success'])
#     fail_count = len(results) - success_count
#     
#     print(f"\n🎉 마이그레이션 완료!")
#     print(f"  - 성공: {success_count}개")
#     print(f"  - 실패: {fail_count}개")
#     
#     # 실패한 파일 목록
#     if fail_count > 0:
#         print("\n❌ 실패한 파일들:")
#         for result in results:
#             if not result['success']:
#                 print(f"  - {result['file_name']}: {result['error']}")
# else:
#     print("❌ 마이그레이션할 파일이 없습니다.")


In [ ]:
# 회사 계정 파일 확인
print("🔍 회사 계정 파일을 확인합니다...")
work_files = scan_drive_files(work_service, 'root', '회사 계정')

if work_files:
    df_work = pd.DataFrame(work_files)
    
    print(f"\n📊 회사 계정 파일 요약:")
    print(f"  - 총 파일 수: {len(df_work)}")
    print(f"  - 폴더 수: {df_work['isFolder'].sum()}")
    print(f"  - 파일 수: {(~df_work['isFolder']).sum()}")
    
    if not df_work['isFolder'].all():
        total_size = df_work[~df_work['isFolder']]['size'].sum()
        print(f"  - 총 크기: {format_file_size(total_size)}")
    
    # 최근 생성된 파일 확인
    recent_files = df_work.head(10)
    print(f"\n📋 최근 파일들:")
    for _, file in recent_files.iterrows():
        print(f"  - {file['name']} ({file['mimeType']})")
else:
    print("❌ 회사 계정에서 파일을 찾을 수 없습니다.")


## 🎉 마이그레이션 완료!

### 다음 단계:
1. **회사 계정에서 파일들이 올바르게 복사되었는지 확인**
2. **공유 설정이 필요한 파일들에 대해 권한 재설정**
3. **개인 계정의 파일들을 정리 (필요시)**
4. **팀원들과 공유 폴더 설정**

### 주의사항:
- **개인 계정의 파일을 삭제하기 전에 충분히 확인하세요**
- **중요한 파일은 백업을 만들어두세요**
- **공유 권한이 필요한 파일들은 수동으로 재설정하세요**

### 문제가 발생했다면:
- 마이그레이션 로그를 확인하세요
- 실패한 파일들을 다시 시도해보세요
- 관리자에게 문의하세요
